# Limpeza dos dados

Neste notebook estão algumas operações para a limpeza dos dados obtidos.

Global Biotic Interactions - Os dados limpos para as interações podem ser encontrados nesse [link](https://drive.google.com/file/d/1YhU8QPXXilg4icozIvttXu2hyfm_0yOC/view?usp=sharing)

Red List - Os dados limpos podem ser encontrados nesse [link](https://drive.google.com/file/d/19c4ObCxYTxyAP99bOWejaXKKrvbN8_3M/view?usp=sharing)

## Interações (Global Biotic Interactions)

Esses dados são fornecidos na forma de um arquivo csv contendo em forma de pares as interações entre diferentes seres no meio ambiente. Como o arquivo fornecido inicialmente era muito grande (aproximadamente 14 GB) para possibilitar o manuseio de forma prática, o primeiro passo adotado foi excluir algumas colunas extras contidas no arquivo, mantendo somente os dados referentes aos animais e o tipo de interação que havia entre eles. Para isso, foi utilizada a ferramenta `miller` extrair somente as colunas desejadas do csv. Com o arquivo reduzido para conter somente as informações necessárias, é possível carregar os dados resultantes da seguinte maneira:

In [1]:
import pandas as pd

interactions_df = pd.read_csv('../../interactions.csv')
interactions_df

,sourceTaxonSpeciesName,sourceTaxonKingdomName,interactionTypeName,targetTaxonSpeciesName,targetTaxonKingdomName
0,Andrena milwaukeensis,Animalia,visitsFlowersOf,Zizia aurea,Plantae
1,Andrena mandibularis,Animalia,visitsFlowersOf,Zanthoxylum americanum,Plantae
2,Andrena edwardsi,Animalia,visitsFlowersOf,Wyethia mollis,Plantae
3,Andrena mandibularis,Animalia,visitsFlowersOf,Viburnum dentatum,Plantae
4,Andrena milwaukeensis,Animalia,visitsFlowersOf,Viburnum lentago,Plantae
...,...,...,...,...,...
7822665,Calyptra orthograpta,Animalia,eats,Cervus unicolor,Animalia
7822666,Calyptra orthograpta,Animalia,eats,Elephas maximus,Animalia
7822667,Calyptra parva,Animalia,eats,NaN,Animalia
7822668,Calyptra pseudobicolor,Animalia,eats,Homo sapiens,Animalia


No entanto, esses dados ainda contém linhas com informações faltantes, o que pode não ser tão útil. Então, para limpar essa tabela foram removidas todas as linhas contendo dados faltantes nas colunas que indicam o nome científico dos animais que participam da interação e a classificação da interação que ocorre entre eles.

In [2]:
interactions_df = interactions_df.dropna(subset=['sourceTaxonSpeciesName', 'interactionTypeName', 'targetTaxonSpeciesName'])
interactions_df

,sourceTaxonSpeciesName,sourceTaxonKingdomName,interactionTypeName,targetTaxonSpeciesName,targetTaxonKingdomName
0,Andrena milwaukeensis,Animalia,visitsFlowersOf,Zizia aurea,Plantae
1,Andrena mandibularis,Animalia,visitsFlowersOf,Zanthoxylum americanum,Plantae
2,Andrena edwardsi,Animalia,visitsFlowersOf,Wyethia mollis,Plantae
3,Andrena mandibularis,Animalia,visitsFlowersOf,Viburnum dentatum,Plantae
4,Andrena milwaukeensis,Animalia,visitsFlowersOf,Viburnum lentago,Plantae
...,...,...,...,...,...
7822664,Calyptra orthograpta,Animalia,eats,Bubalus bubalis,Animalia
7822665,Calyptra orthograpta,Animalia,eats,Cervus unicolor,Animalia
7822666,Calyptra orthograpta,Animalia,eats,Elephas maximus,Animalia
7822668,Calyptra pseudobicolor,Animalia,eats,Homo sapiens,Animalia


In [3]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3835705 entries, 0 to 7822669
Data columns (total 5 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   sourceTaxonSpeciesName  object
 1   sourceTaxonKingdomName  object
 2   interactionTypeName     object
 3   targetTaxonSpeciesName  object
 4   targetTaxonKingdomName  object
dtypes: object(5)
memory usage: 175.6+ MB


O nome das espécies e dos reinos nem sempre é escrito da mesma forma e as vezes contém erros, como espaços duplos, então é preciso corrigí-los.

In [4]:
import re
def normalize_species_name(s):
    s = re.sub('\(.*\)', '', s) # Remove parentese
    s = s.lower() # Passa pra minusculo
    s = s.replace('[?�]', '') # Remove ?
    s = re.sub('\s+', ' ', s) # Usa sempre apenas 1 espaços
    s = re.sub('= .* ', '', s) # Remove termos da forma =PALAVRA 
    s = re.sub('= .*,', ',', s) # Remove termos da forma =PALAVRA,
    s = s.strip() # Remove espaços no começo e fim
    return s
interactions_df.sourceTaxonSpeciesName = interactions_df.sourceTaxonSpeciesName.apply(normalize_species_name)
interactions_df.targetTaxonSpeciesName = interactions_df.targetTaxonSpeciesName.apply(normalize_species_name)

interactions_df.replace(
    'tetrabothrius; tetrabothrius tetrabothrius jagerskioeldi; tetrabothrius jagerskioeldi; tetrabothrius jaegerskioeldi; tetrabothrius jagerskioeldi',
    'tetrabothrius jagerskioeldi',
    inplace=True)
interactions_df.replace(
    'Animalia; Animalia',
    'Animalia',
    inplace=True)
interactions_df

/usr/lib/python3.7/site-packages/pandas/core/generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/lib/python3.7/site-packages/pandas/core/frame.py:4527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,sourceTaxonSpeciesName,sourceTaxonKingdomName,interactionTypeName,targetTaxonSpeciesName,targetTaxonKingdomName
0,andrena milwaukeensis,Animalia,visitsFlowersOf,zizia aurea,Plantae
1,andrena mandibularis,Animalia,visitsFlowersOf,zanthoxylum americanum,Plantae
2,andrena edwardsi,Animalia,visitsFlowersOf,wyethia mollis,Plantae
3,andrena mandibularis,Animalia,visitsFlowersOf,viburnum dentatum,Plantae
4,andrena milwaukeensis,Animalia,visitsFlowersOf,viburnum lentago,Plantae
...,...,...,...,...,...
7822664,calyptra orthograpta,Animalia,eats,bubalus bubalis,Animalia
7822665,calyptra orthograpta,Animalia,eats,cervus unicolor,Animalia
7822666,calyptra orthograpta,Animalia,eats,elephas maximus,Animalia
7822668,calyptra pseudobicolor,Animalia,eats,homo sapiens,Animalia


Além disso, os dados obtidos podem ter ocorrências repetidas. Para evitar recontar uma mesma interação entre dois animais, serão removidas todas as interações que contenham a mesma fonte, o mesmo alvo e a mesma ação.

In [5]:
interactions_df = interactions_df.drop_duplicates(subset=['sourceTaxonSpeciesName', 'interactionTypeName', 'targetTaxonSpeciesName'], keep="first")
interactions_df

,sourceTaxonSpeciesName,sourceTaxonKingdomName,interactionTypeName,targetTaxonSpeciesName,targetTaxonKingdomName
0,andrena milwaukeensis,Animalia,visitsFlowersOf,zizia aurea,Plantae
1,andrena mandibularis,Animalia,visitsFlowersOf,zanthoxylum americanum,Plantae
2,andrena edwardsi,Animalia,visitsFlowersOf,wyethia mollis,Plantae
3,andrena mandibularis,Animalia,visitsFlowersOf,viburnum dentatum,Plantae
4,andrena milwaukeensis,Animalia,visitsFlowersOf,viburnum lentago,Plantae
...,...,...,...,...,...
7822650,calyptra orthograpta,Animalia,eats,bubalus bubalis,Animalia
7822651,calyptra orthograpta,Animalia,eats,cervus unicolor,Animalia
7822652,calyptra orthograpta,Animalia,eats,elephas maximus,Animalia
7822654,calyptra pseudobicolor,Animalia,eats,homo sapiens,Animalia


Outro ponto a se atentar é que algumas espécies são marcadas como **não identificadas**. Como isso não contribuirá diretamente para a análise, interações com espécies não identificadas serão removidas.

In [6]:
unidentified_source = interactions_df.sourceTaxonSpeciesName.str.contains('unidentified')
interactions_df = interactions_df[~unidentified_source]

unknown_source = interactions_df.sourceTaxonSpeciesName.str.contains('unknown')
interactions_df = interactions_df[~unknown_source]

na_source = interactions_df.sourceTaxonSpeciesName.str.contains('n/a n/a')
interactions_df = interactions_df[~na_source]

num_no_source = sum(map(sum, [unidentified_source, unknown_source, na_source]))
print('Total de interações onde a fonte não foi identificada:', num_no_source)


unidentified_target = interactions_df.targetTaxonSpeciesName.str.contains('unidentified')
interactions_df = interactions_df[~unidentified_target]

unknown_target = interactions_df.targetTaxonSpeciesName.str.contains('unknown')
interactions_df = interactions_df[~unknown_target]

na_target = interactions_df.targetTaxonSpeciesName.str.contains('n/a n/a')
interactions_df = interactions_df[~na_target]

num_no_target = sum(map(sum, [unidentified_target, unknown_target, na_target]))
print('Total de interações onde o alvo não foi identificado:', num_no_target)

interactions_df

Total de interações onde a fonte não foi identificada: 113
Total de interações onde o alvo não foi identificado: 4047


,sourceTaxonSpeciesName,sourceTaxonKingdomName,interactionTypeName,targetTaxonSpeciesName,targetTaxonKingdomName
0,andrena milwaukeensis,Animalia,visitsFlowersOf,zizia aurea,Plantae
1,andrena mandibularis,Animalia,visitsFlowersOf,zanthoxylum americanum,Plantae
2,andrena edwardsi,Animalia,visitsFlowersOf,wyethia mollis,Plantae
3,andrena mandibularis,Animalia,visitsFlowersOf,viburnum dentatum,Plantae
4,andrena milwaukeensis,Animalia,visitsFlowersOf,viburnum lentago,Plantae
...,...,...,...,...,...
7822650,calyptra orthograpta,Animalia,eats,bubalus bubalis,Animalia
7822651,calyptra orthograpta,Animalia,eats,cervus unicolor,Animalia
7822652,calyptra orthograpta,Animalia,eats,elephas maximus,Animalia
7822654,calyptra pseudobicolor,Animalia,eats,homo sapiens,Animalia


Algumas espécies têm registradas interações com elas mesmas, mas essas interações não contribuem para nossa análise e são removidas.

In [7]:
self_interactions = interactions_df.query('sourceTaxonSpeciesName == targetTaxonSpeciesName')
print("Total de interações com a mesma espécie: {}".format(len(self_interactions)))

interactions_df = interactions_df.drop(self_interactions.index)
interactions_df

Total de interações com a mesma espécie: 22603


,sourceTaxonSpeciesName,sourceTaxonKingdomName,interactionTypeName,targetTaxonSpeciesName,targetTaxonKingdomName
0,andrena milwaukeensis,Animalia,visitsFlowersOf,zizia aurea,Plantae
1,andrena mandibularis,Animalia,visitsFlowersOf,zanthoxylum americanum,Plantae
2,andrena edwardsi,Animalia,visitsFlowersOf,wyethia mollis,Plantae
3,andrena mandibularis,Animalia,visitsFlowersOf,viburnum dentatum,Plantae
4,andrena milwaukeensis,Animalia,visitsFlowersOf,viburnum lentago,Plantae
...,...,...,...,...,...
7822650,calyptra orthograpta,Animalia,eats,bubalus bubalis,Animalia
7822651,calyptra orthograpta,Animalia,eats,cervus unicolor,Animalia
7822652,calyptra orthograpta,Animalia,eats,elephas maximus,Animalia
7822654,calyptra pseudobicolor,Animalia,eats,homo sapiens,Animalia


Por fim, os dados são salvos mais uma vez em disco.

In [8]:
interactions_df.to_csv('../../clean-interactions.csv', index=False)

## Red list (IUCN)

Para os dados da Red List, as informações sobre a condição das espécies e as suas localizações foram obtidas de forma separada. Assim, o primeiro passo foi carregar os dados sobre a condição das populações de cada espécie e excluir as colunas com dados que não serão utilizados, como versão do critério ou a língua na qual o critério foi publicado.

In [9]:
import pandas as pd

redlist_df = pd.read_csv('../../assessments.csv')
redlist_df = redlist_df[['scientificName', 'redlistCategory', 'populationTrend', 'systems', 'realm', 'possiblyExtinct', 'possiblyExtinctInTheWild']]
redlist_df

,scientificName,redlistCategory,populationTrend,systems,realm,possiblyExtinct,possiblyExtinctInTheWild
0,Hexanchus griseus,Near Threatened,Decreasing,Marine,NaN,False,False
1,Heosemys annandalii,Critically Endangered,Decreasing,Terrestrial|Freshwater (=Inland waters),Indomalayan,False,False
2,Hubbsina turneri,Critically Endangered,Decreasing,Freshwater (=Inland waters),Neotropical,False,False
3,Hungerfordia pelewensis,Endangered,Unknown,Terrestrial,Oceanian,False,False
4,Ictalurus mexicanus,Vulnerable,Unknown,Freshwater (=Inland waters),Neotropical,False,False
...,...,...,...,...,...,...,...
50245,Filicium thouarsianum,Near Threatened,Decreasing,Terrestrial,Afrotropical,False,False
50246,Melanophylla angustior,Endangered,Decreasing,Terrestrial,Afrotropical,False,False
50247,Melanophylla madagascariensis,Endangered,Decreasing,Terrestrial,Afrotropical,False,False
50248,Melanophylla modestei,Endangered,Decreasing,Terrestrial,Afrotropical,False,False


Em seguida, carregaram-se os dados de localização das espécies, de onde foram extraídas as colunas contendo o nome científico das espécies e a latitude e longitude da sua localização. Como algumas espécies podem ser encontradas em mais de uma posição, todas as coordenadas foram mantidas na forma de uma lista.

In [10]:
locations_df = pd.read_csv('../../points_data.csv')
locations_df = locations_df[['binomial', 'longitude', 'latitude']]
locations_df = locations_df.groupby(by='binomial').agg(lambda x: list(x))
locations_df

,longitude,latitude
binomial,,
Abantis bicolor,"[31.44849968, 31.7266674, 29.51413918, 31.7266...","[-28.89819527, -28.83063889, -31.607584, -28.8..."
Abarema callejasii,"[-75.57151548, -75.49367, -75.0166667, -75.016...","[6.92677292, 7.06056, 6.0166667, 6.9, 7.1, 4.7..."
Abarema josephi,"[-75.67611, -75.5477557, -75.3636, -75.6625, -...","[1.85694444, 6.1603885, 6.1719, 6.2325, 6.1097..."
Abatia mexicana,"[-98.040278, -97.02715158, -97.285, -97.220555...","[20.124444, 19.430496, 19.841389, 19.830833, 1..."
Abrahamia deflexa,"[46.17361, 45.25, 47.233333, 45.78333, 46.8166...","[-15.71028, -16.83333, -15.066667, -15.93333, ..."
...,...,...
Zygia lehmannii,"[-77.5048108, -76.82328, -76.55524, -76.52067,...","[2.7022732, 2.89315, 2.56236, 2.54989, 1.17404..."
Zygia multipunctata,"[-75.0505556, -75.06391, -75.0505556, -75.0666...","[7.2166667, 7.32045, 7.2166667, 7.3, 7.3037070..."
Zygogynum tanyostigma,"[164.76840651, 164.76666667, 164.79041, 164.79...","[-20.5824393, -20.5833333, -20.566, -20.566, -..."


Após isso, a informação da localização das espécies foi combinada ao restante da tabela, resultando no seguinte:

In [11]:
redlist_df = redlist_df.join(locations_df, on='scientificName')
redlist_df

,scientificName,redlistCategory,populationTrend,systems,realm,possiblyExtinct,possiblyExtinctInTheWild,longitude,latitude
0,Hexanchus griseus,Near Threatened,Decreasing,Marine,NaN,False,False,NaN,NaN
1,Heosemys annandalii,Critically Endangered,Decreasing,Terrestrial|Freshwater (=Inland waters),Indomalayan,False,False,NaN,NaN
2,Hubbsina turneri,Critically Endangered,Decreasing,Freshwater (=Inland waters),Neotropical,False,False,"[-101.4795, -101.7773, -101.7876]","[19.8745, 19.8256, 19.8273]"
3,Hungerfordia pelewensis,Endangered,Unknown,Terrestrial,Oceanian,False,False,NaN,NaN
4,Ictalurus mexicanus,Vulnerable,Unknown,Freshwater (=Inland waters),Neotropical,False,False,"[-99.35417, -99.3]","[21.98083, 22.0]"
...,...,...,...,...,...,...,...,...,...
50245,Filicium thouarsianum,Near Threatened,Decreasing,Terrestrial,Afrotropical,False,False,"[49.13333, 49.066667, 49.13333, 49.13333, 47.7...","[-18.81666, -18.866667, -18.81666, -18.81666, ..."
50246,Melanophylla angustior,Endangered,Decreasing,Terrestrial,Afrotropical,False,False,"[47.72611, 47.7322]","[-23.01306, -23.0219]"
50247,Melanophylla madagascariensis,Endangered,Decreasing,Terrestrial,Afrotropical,False,False,"[49.2001111, 49.20305, 49.20333, 49.20388, 49....","[-17.9206389, -17.93194, -17.93194, -17.92944,..."
50248,Melanophylla modestei,Endangered,Decreasing,Terrestrial,Afrotropical,False,False,"[49.98333, 49.975, 49.975, 49.96666, 49.966667...","[-15.63333, -15.625, -15.625, -15.63333, -15.6..."


A mesma normalização deve ser aplicada as espécies dessa base de dados para casar com a de interações

In [12]:
redlist_df.scientificName = redlist_df.scientificName.apply(normalize_species_name)
redlist_df

,scientificName,redlistCategory,populationTrend,systems,realm,possiblyExtinct,possiblyExtinctInTheWild,longitude,latitude
0,hexanchus griseus,Near Threatened,Decreasing,Marine,NaN,False,False,NaN,NaN
1,heosemys annandalii,Critically Endangered,Decreasing,Terrestrial|Freshwater (=Inland waters),Indomalayan,False,False,NaN,NaN
2,hubbsina turneri,Critically Endangered,Decreasing,Freshwater (=Inland waters),Neotropical,False,False,"[-101.4795, -101.7773, -101.7876]","[19.8745, 19.8256, 19.8273]"
3,hungerfordia pelewensis,Endangered,Unknown,Terrestrial,Oceanian,False,False,NaN,NaN
4,ictalurus mexicanus,Vulnerable,Unknown,Freshwater (=Inland waters),Neotropical,False,False,"[-99.35417, -99.3]","[21.98083, 22.0]"
...,...,...,...,...,...,...,...,...,...
50245,filicium thouarsianum,Near Threatened,Decreasing,Terrestrial,Afrotropical,False,False,"[49.13333, 49.066667, 49.13333, 49.13333, 47.7...","[-18.81666, -18.866667, -18.81666, -18.81666, ..."
50246,melanophylla angustior,Endangered,Decreasing,Terrestrial,Afrotropical,False,False,"[47.72611, 47.7322]","[-23.01306, -23.0219]"
50247,melanophylla madagascariensis,Endangered,Decreasing,Terrestrial,Afrotropical,False,False,"[49.2001111, 49.20305, 49.20333, 49.20388, 49....","[-17.9206389, -17.93194, -17.93194, -17.92944,..."
50248,melanophylla modestei,Endangered,Decreasing,Terrestrial,Afrotropical,False,False,"[49.98333, 49.975, 49.975, 49.96666, 49.966667...","[-15.63333, -15.625, -15.625, -15.63333, -15.6..."


Por fim, a tabela com os dados organizados foi salva novamente em disco.

In [13]:
redlist_df.to_csv('../../clean-redlist.csv')